# [Advent of Code 2017](http://adventofcode.com/2017)

In previous years I've done Advent in Code in Go (2015) and Elixir (2016).
In 2016 I compared solutions with [Cameron](https://github.com/wheat779).

He was completing the questions in Python which required I brush up on Python to be able to assist him with improving his solutions as it had been a number of years since I had touched the language.

With the rise of Python in the Data Science and Machine Learning space I've decided to use Python as my language of choice this year. I'm completing my answers in a Notebook to document the thought processes behind my answers.

To warm up for 2017 I've completed a few of the 2016 questions in Python and begun translating a number of general helper functions to Python from my Elixir answers last year as well as some [Project Euler](https://projecteuler.net) helper functions that *may* come in handy.

I've also stolen and modified a few simple functions from [Peter Norvig's AoC 2016 Notebook](https://github.com/norvig/pytudes/blob/master/ipynb/Advent%20of%20Code.ipynb).

I've placed these helpers in a Notebook named `AoC_Helpers.ipynb` to allow them to be shared between my 2015, 2016 and 2017 Notebooks.

Here, I use the [ipnyb](https://github.com/ipython/ipynb) module to import that Notebook as if it were a normal Python module.

In [1]:
from ipynb.fs.full.aoc_helpers import *

When doing Project Euler I've implemented plenty of graphs and prime sieves so I'll be using more efficient libraries for these in case they come up.

In [2]:
# ! pip install networkx primesieve
import networkx
import primesieve
from numba import jit

In [3]:
import re
import numpy as np
import math
import itertools as it
import operator


from collections import Counter, defaultdict, namedtuple, deque
from functools   import lru_cache, partial,reduce
from itertools   import permutations, combinations, chain, cycle, product, islice
from heapq       import heappop, heappush

As the Helpers module is shared between 2015, 2016 and 2017, I'm partially applying the `Input` function to specify the year.

`Input(day, year) -> partial(Input, year=2017) -> Input(3) == Input(3, 2017)`

This allows me to use `Input(3)` in the 2016 file to get day 3 of 2016, as well as `Input(3)` in the 2017 file to get day 3 of 2017.

In [4]:
Input = partial(Input, year=2017)

## [Day 1](http://adventofcode.com/2017/day/1): Inverse Captcha

Wooo! Time to kick of Advent of Code 2017!

For Part A we need to sum all digits that match the **next digit** in the list. The list is circular, with the end of the list wrapping around to the front.

This means if the front of the list is a 3 and the end of the list is a 3, the end of the list will consider the next digit to be a 3 and thus be added to the sum.

### Part A

In order to compare the value of index $ n $ in the array with $ n + step $ we can create a copy of the array, offset by $ step $. This solves the issue with wrapping around the end of the array and visually is cleaner than indexing.

We then use `zip` to get a tuple of each element of the array along with the offset array to easily compare the two.

In [5]:
one = read_letters(Input(1))

In [6]:
def captcha(nums, part_b=False):
    step = len(nums) // 2 if part_b else 1
    return sum(int(a) for a, b in zip(nums, nums[step:] + nums[:step]) if a == b)

In [7]:
assert captcha('1221') == 3
assert captcha('1111') == 4
assert captcha('1234') == 0
assert captcha('91212129') == 9

In [8]:
captcha(one)

1141

### Part B
For Part B, instead of considering the next number we want to consider the digit **halfway around** the circular list.

I've added a `part_b` argument which changes the `step` size from 1 to half the length of the input.

In [10]:
assert captcha('1212', part_b=True) == 6
assert captcha('1221', part_b=True) == 0
assert captcha('123425', part_b=True) == 4
assert captcha('123123', part_b=True) == 12
assert captcha('12131415', part_b=True) == 4

In [11]:
captcha(one, part_b=True)

950

In case more array rotations end up coming up in the coming days I'm going to define a function to allow me to easily rotate an array a given number of steps.

In [12]:
def rotate(iterable, steps=1):
    "Rotate the iterable a given number of steps."
    steps = steps % len(iterable)
    return iterable[steps:] + iterable[:steps] 

In [13]:
assert rotate([1, 2, 3, 4, 5]) == [2, 3, 4, 5, 1]
assert rotate([1, 2, 3, 4, 5], steps=-1) == [5, 1, 2, 3, 4]
assert rotate([1, 2, 3, 4, 5], steps=3)  == [4, 5, 1, 2, 3]
assert rotate([1, 2, 3, 4, 5], steps=10) == [1, 2, 3, 4, 5]

## [Day 2](http://adventofcode.com/2017/day/2): Corruption Checksum

At first glance this seems even more straightforward than day 1.

Given a tab separated spreadsheet we need to compute a checksum!

For each row, find the $ min $ and $ max $ values and compute the difference between them.

The checksum is the sum of the result of each row.

In [14]:
def int_lines(lines):
    "Parse ints of each subarray of the input array"
    return list(map(parse_ints, lines))

In [15]:
spreadsheet = int_lines(Input(2))

The difference between the $ min $ and $ max $ amplitude of a waveform is known as the Peak to Peak value.

In [16]:
def ptp(iterable):
    "The difference between the maximum and minimum values"
    return max(iterable) - min(iterable)

In [17]:
assert ptp([5, 1, 9, 5]) == 8

In [18]:
sum(ptp(row) for row in spreadsheet)

41887

### Part B

Instead of summing the min and max values from each row we now must find the two numbers for which $a \bmod b = 0$, keeping the result of $a\div b$

In [19]:
def evenly_divisible(ints):
    "Return the evenly divisible combinations of numbers in the input"
    for (a, b) in combinations(sorted(ints), 2):
        if b % a == 0: yield (b, a)

In [20]:
assert next(evenly_divisible([5, 9, 2, 8])) == (8, 2)

In [21]:
sum(a // b for row in spreadsheet for (a, b) in evenly_divisible(row))

226

## [Day 3](http://adventofcode.com/2017/day/3): Spiral Memory

You come across an experimental new kind of memory stored on an infinite two-dimensional grid.

Each square on the grid is allocated in a spiral pattern starting at a location marked 1 and then counting up while spiraling outward. For example, the first few squares are allocated like this:

```
7   16  15  14  13
18   5   4   3  12
19   6   1   2  11
20   7   8   9  10
21  22  23---> ...
```

Observing the number of spaces moved before each turn we can see a clear pattern:

```
4   4   4   4  3
4   2   2   1  3
4   2   0   1  3
4   2   3   3  3
4   5   5--> ...
```

First, we move one space to the right. We then turn to the left and move one space upwards.  
Now we turn and move two spaces to the left. We turn, and move two spaces.  
We turn, and move three spaces. We turn, and move three spaces.  
We turn, and move four spa...  

For each iteration of the spiral we move forward $ n $ spaces for two turns, we then move $ n+1 $ spaces for two turns, incrementing $ n $ by $ 1 $ every two turns.


In [22]:
# The length of a side is the first odd square larger than our goal
def find_side_length(goal):
    return next(n for n in it.count(1, 2) if n**2 >= goal)

def spiral_walk_some(goal):
    # First, find the length of the ring we're on
    side_length = find_side_length(goal)
    # The ring number is the floor of length / 2
    ring = side_length // 2
    # Easy enough to get the bottom right corner value based off the length
    value = side_length**2
    # So now we have a location and a value.
    position = Point(ring, ring)
    
    # Walk until we get to the value we're after
    for dir in [Point(-1,0), Point(0,-1), Point(1,0), Point(0,1)]:
        for _ in range(side_length-1):
            if value == goal:
                return cityblock_distance(position)
            position += dir
            value -= 1
    return 0

In [23]:
def spiral_cameron(goal):
    n = 1
    while n * n < goal:
        n += 1
    steps = (n - 1) / 2

    down = n * n - (n - 1) / 2
    left = down - (n - 1)
    up = left - (n - 1)
    right = up - (n - 1)

    possible_paths = [goal - down, goal - left, goal - up, goal - right]

    steps += abs(min(*possible_paths));
    return int(steps)

In [24]:
goal = 347991

In [25]:
%timeit spiral_walk_some(goal)

1.88 ms ± 191 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [26]:
%timeit spiral_cameron(goal)

73.9 µs ± 1.98 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [27]:
def spiral(part_b=False):
    matrix, position, facing = defaultdict(int), Point(0, 0), Point(1, 0)
    max_steps, stepped, turn_count, num = 1, 0, 0, 0

    while True:
        if part_b:
            num = sum(matrix[n] for n in neighbors8(position))
            if not num: num = 1
        else:
            num += 1

        matrix[position] = num
        yield (position, num)

        position += facing
        stepped += 1
        
        if stepped == max_steps:
            stepped = 0
            turn_count += 1
            # We increase the distance traveled every two turns
            if turn_count == 2:
                max_steps += 1
                turn_count = 0
            # Rotate to the left by swapping x and y and negating one of them.
            facing = Point(facing.y, -facing.x)

### Part A
For Part A we're asked to find the cityblock distance from 0,0 to the point that holds our input value of 347991

In [28]:
def spiral_distance(n): return next(cityblock_distance(pos) for (pos, val) in spiral() if val == n)

In [29]:
assert spiral_distance(1) == 0
assert spiral_distance(12) == 3
assert spiral_distance(23) == 2
assert spiral_distance(1024) == 31

In [30]:
spiral_distance(goal)

480

### Part B

For Part B we modify how values are calculated. Instead of the spiral being a monotonically increasing value we instead sum the values of all of the squares neighbors at the time of calculation.  
I've added a `part_b` argument to the `spiral` generator to modify its number generation to follow the required format.

```
147  142  133  122   59
304    5    4    2   57
330   10    1    1   54
351   11   23   25   26
362  747  806--->   ...
```

We're asked to find the value in the spiral that immediately follows our input value.

In [31]:
def next_spiral_value(n): return next(val for (pos, val) in spiral(part_b=True) if val > n)

In [32]:
assert next_spiral_value(23) == 25
assert next_spiral_value(308) == 330
assert next_spiral_value(482) == 747

In [33]:
next_spiral_value(goal)

349975

## [Day 4](http://adventofcode.com/2017/day/4): High-Entropy Passphrases


### Part A

To ensure security, a valid passphrase must contain no duplicate words.

For example:

`aa bb cc dd ee` is valid.  
`aa bb cc dd aa` is not valid - the word aa appears more than once.  
`aa bb cc dd aaa` is valid - aa and aaa count as different words.  

In [34]:
passphrases = read_words(Input(4))

The easiest way to determine that every word in a list is unique is to create a set from the list and compare the lengths.

In [35]:
def all_unique(iterable):
    return len(iterable) == len(set(iterable))

In [36]:
sum(1 for phrase in passphrases if all_unique(phrase))

455

### Part B

For part B a passphrase must contain no duplicate anagrams.

`abcde fghij` is a valid passphrase.  
`abcde xyz ecdab` is not valid - the letters from the third word can be rearranged to form the first word.  
`a ab abc abd abf abj` is a valid passphrase, because all letters need to be used when forming another word.  
`iiii oiii ooii oooi oooo` is valid.  
`oiii ioii iioi iiio` is not valid - any of these words can be rearranged to form any other word.  


If we sort each word in the passphrase we'll be able to easily detect anagrams.

In [37]:
def sort_word(word):
    return cat(sorted(word))

In [38]:
sorted_passphrases = [[sort_word(word) for word in phrase] for phrase in passphrases]

In [39]:
sum(1 for phrase in sorted_passphrases if all_unique(phrase)) 

186

## [Day 5](http://adventofcode.com/2017/day/5): A Maze of Twisty Trampolines, All Alike

Welcome to the wonderful world of relative `GOTO` statements.  
Each instruction is a number of instructions which we will jump ahead (if positive) or behind (if negative) our curent position.  
Before we jump to the next instruction we increment the value of our current position by 1.


Given the input: `0  3  0  1  -3`
```
(0) 3  0  1  -3  - before we have taken any steps.
(1) 3  0  1  -3  - jump with offset 0 (that is, don't jump at all). Fortunately, the instruction is then incremented to 1.
 2 (3) 0  1  -3  - step forward because of the instruction we just modified. The first instruction is incremented again, now to 2.
 2  4  0  1 (-3) - jump all the way to the end; leave a 4 behind.
 2 (4) 0  1  -2  - go back to where we just were; increment -3 to -2.
 2  5  0  1  -2  - jump 4 steps forward, escaping the maze.
 ```
 
In this example, the exit is reached in **5** steps.


### Part A

How many steps does it take to reach the exit?

In [40]:
day5 = list(map(int, Input(5)))

In [170]:
@jit
def goto_escape(instructions, part_b=False):
    instructions = instructions.copy()
    location, moves = 0, 0
    
    while location < len(instructions):
        jump = instructions[location]
        if part_b and jump >= 3:
            instructions[location] -= 1
        else:
            instructions[location] += 1
        location += jump
        moves += 1
    return moves

In [178]:
goto_escape(day5)

358309

### Part B

Now, the jumps are even stranger: after each jump, if the offset was three or more, instead decrease it by 1. Otherwise, increase it by 1 as before.

In [179]:
goto_escape(day5, part_b=True)

28178177

## [Day 6](http://adventofcode.com/2017/day/6): Memory Reallocation

In [46]:
day6 = [int(x) for x in Input(6).read().strip().split('\t')]

In [47]:
def reallocate_original(banks, part_b=False):
    banks = banks.copy()
    seen = {}
    steps = 0

    while tuple(banks) not in seen:
        seen[tuple(banks)] = steps
        steps += 1
        # Get the bank that needs to be redistributed
        max_count = max(banks)
        max_index = banks.index(max_count)
        # Clear it and begin rellocating the memory
        banks[max_index] = 0
        for i in range(1, max_count + 1):
            banks[(max_index + i) % len(banks)] += 1

    if part_b:
        return steps - seen[tuple(banks)]
    return steps

Now, this method worked fine for solving the actual problem but the actual cycle detection could be made more generic, in case we encounter the need on another day.

To do this we'll first extract the actual reallocation step into its own function

In [48]:
def reallocate(banks):
    "Redistribute the highest valued bank"
    max_count = max(banks)
    max_index = banks.index(max_count)
    banks[max_index] = 0
    for i in range(1, max_count + 1):
        banks[(max_index + i) % len(banks)] += 1
    return banks

In [49]:
def find_cycle(initial_state, func, limit=100_000):
    "Given an initial state and a function that transforms the state find a cycle within `limit` steps."
    if isinstance(initial_state, list):
        state, serializer = initial_state.copy(), tuple
    else:
        state, serializer = initial_state, lambda x: x
    seen, steps = {}, 0

    while serializer(state) not in seen:
        seen[serializer(state)] = steps
        steps += 1
        if limit and steps >= limit:
            return f'No cycle found in limit of {limit} steps'
        state = func(state)

    first_seen = seen[serializer(state)]
    return {
        'cycle_item': state,
        'total_steps': steps, 
        'first_seen': first_seen,
        'cycle_length': steps - first_seen
    }

In [50]:
find_cycle([0, 2, 7, 0], reallocate) 

{'cycle_item': [2, 4, 1, 2],
 'cycle_length': 4,
 'first_seen': 1,
 'total_steps': 5}

In [180]:
find_cycle(day6, reallocate)

{'cycle_item': [0, 14, 13, 12, 11, 10, 8, 8, 6, 6, 5, 3, 3, 2, 1, 10],
 'cycle_length': 2392,
 'first_seen': 4289,
 'total_steps': 6681}

## [Day 7](http://adventofcode.com/2017/day/7): Recursive Circus

In [52]:
in7 = Input(7).readlines()

In [53]:
def replace_all(chars, replacement, text):
    "Replace each instance of each individual char with the replacement text"
    for char in chars:
        text = text.replace(char, replacement)
    return text

In [54]:
def parse7(line):
    node, weight, *children = replace_all('(),->', '', line).split()
    return (node, int(weight), children)

In [55]:
day7 = [parse7(line) for line in in7]

Fastest way I can think of finding the root node is creating a set of nodes that have children as well as nodes which are children.

Do a set subtraction and we'll have our node that has children and isn't a child itself

In [56]:
nodes, is_child = set(), set()

for name, weight, children in day7:
    nodes.add(name)
    for child in children:
        is_child.add(child)
            
root = (nodes - is_child).pop()
root

'gmcrj'

I'm going to try using NetworkX as it seems like having a solid graph library under my belt would be useful.

In [112]:
import networkx as nx

In [185]:
def create_graph(inp):
    G = nx.DiGraph()
    for name, weight, children in inp:
        G.add_node(name, weight=weight, total_weight=-1)   
        for child in children:
            G.add_edge(name, child)
    return G

In [192]:
def sum_children(G, node):
    if G.nodes[node]['total_weight'] != -1:
        return G.nodes[node]['total_weight']
    total_weight = G.nodes[node]['weight']
    total_weight += sum(sum_children(G, child) for child in G.successors(node))
    G.nodes[node]['total_weight'] = total_weight
    return total_weight

In [193]:
def add_total_weights(G):
    for node in nx.topological_sort(G):
        sum_children(G, node)

In [194]:
G = create_graph(day7)
add_total_weights(G)

Networkx topoligical sort orders the nodes from the root node.

In [207]:
nodelist = list(nx.topological_sort(G))

In [208]:
nodelist[0]

'gmcrj'

In [202]:
list(G.successors('gmcrj'))

['polkn', 'gejdtfw', 'cfuudsk', 'fqoary']

In [105]:
weights = {}
children = {}
parents = {}

for name, weight, kids in day7:
    weights[name] = weight
    children[name] = kids
    for kid in kids:
        parents[kid] = name

In [59]:
from collections import Counter

def take_unique(iterable, key=None):
    "Return an iterable of the unique elements of the input."
    if key is None:
        key = lambda x: x
    seen = Counter([key(item) for item in iterable])
    for item in iterable:
        if seen[key(item)] == 1:
            yield item
            
def first_unique(iterable, key=None, default=None):
    "Return the first unique element of an iterable."
    if key is None:
        key = lambda x: x
    seen = Counter([key(item) for item in iterable])
    for item in iterable:
        if seen[key(item)] == 1:
            return item
    return default

def uniq_comm(iterable, key=None, default=None):
    "Return the unique element as well as the common element of an iterable."
    if key is None:
        key = lambda x: x
    once = set()
    dupes = set()
    for item in iterable:
        if item in once:
            once.remove(item)
            dupes.add(item)
        if item in dupes:
            continue
        if item not in dupes:
            once.add(item)
        if len(once) + len(dupes) > 2:
            break
    if len(once) == 0:
        raise ValueError("No unique value")  
    if len(once) > 1:
        raise ValueError("More than one unique value")
    if len(dupes) == 0:
        raise ValueError("No duplicated value")
    if len(dupes) > 1:
        raise ValueError("More than one duplicated value")
    return (once.pop(), dupes.pop())

In [60]:
assert list(take_unique([1,1])) == []
assert list(take_unique([1,2,3,2,1,200])) == [3, 200]
assert first_unique([1,2,3,2,1]) == 3
assert first_unique([1,1]) == None
assert first_unique([1,1], default=4) == 4

In [61]:
def find_imbalance(node):
    print(node)
    if node in children:
        kids = children[node]
        kid_weights = [(kid, sum_children(kid)) for kid in kids]
        print(kid_weights)
        if len(set([weight for (kid, weight) in kid_weights])) > 1:
            (name, weight) = next(take_unique(kid_weights, key=lambda x: x[1]))
            return find_imbalance(name)
    else:
        print('nah')
        return (node,weights[node] )

In [63]:
#find_imbalance(root)

In [64]:
def sum_children(node):
    if node in children:
        return weights[node] + sum(sum_children(child) for child in children.get(node))
    return weights[node]

In [65]:
[(child, sum_children(child)) for child in children.get('gmcrj')]

[('polkn', 137832),
 ('gejdtfw', 137837),
 ('cfuudsk', 137832),
 ('fqoary', 137832)]

In [66]:
[(child, sum_children(child)) for child in children.get('gejdtfw')]

[('yeoia', 14390),
 ('fbtzaic', 14395),
 ('tbmtw', 14390),
 ('rutvyr', 14390),
 ('tucqq', 14390)]

In [67]:
[(child, sum_children(child)) for child in children.get('fbtzaic')]

[('nzkxl', 913), ('mdbtyw', 918), ('dqwfuzn', 913)]

In [68]:
[(child, sum_children(child)) for child in children.get('mdbtyw')]

[('zqcrxm', 174), ('lfbocy', 174), ('uqrmg', 174)]

In [69]:
weights['mdbtyw'] - 5

391

## [Day 8](http://adventofcode.com/2017/day/8): I Heard You Like Registers

In [70]:
line = Input(8).read().strip()

In [71]:
def parse_instruction(instruction):
    var, op, val, if_, cmp_var, cmp_op, cmp_val = instruction.split()
    op = '-=' if op == 'dec' else '+='
    return f"if registers['{cmp_var}'] {cmp_op} {cmp_val}: registers['{var}'] {op} {val}"

def execute_instructions(instructions, part_b=False):
    # Use a defaultdict so we can easily compare nonexistent keys
    registers = defaultdict(int)
    instructions = [parse_instruction(i) for i in instructions]
    
    # Define a lambda to get the max value out of the dictionary
    get_max_value = lambda x: x[max(x, key=x.get)]
    
    max_seen = 0
    for instruction in instructions:
        # Use pythons exec to execute the instruction
        exec(instruction)
        max_now = get_max_value(registers)
        if max_now > max_seen:
            max_seen = max_now
            
    if part_b:
        return max_seen
    return get_max_value(registers)

In [72]:
def parse_in(ins):
    return ins.replace('dec', '-=').replace('inc','+=').strip() + ' else 0'

def execute_ins(instructions, part_b=False):
    # Use a defaultdict so we can easily compare nonexistent keys
    registers = defaultdict(int)
    instructions = [parse_in(i) for i in instructions]
    get_max_value = lambda x: x[max(x, key=x.get)]
    
    max_seen = 0
    for instruction in instructions:
        # Passing the defaultdict as the locals dictionary causes some interesting properties.
        # when exec attempts to assign to a variable that doesn't exist, its just like a nonexistent dictionary key since
        # technically the local variables is a dictionary lookup. And locals is a defaultdict.
        exec(instruction, globals(), registers)
        max_seen = max(max_seen, get_max_value(registers))
            
    if part_b:
        return max_seen
    return get_max_value(registers)

In [181]:
execute_instructions(Input(8))

4163

In [74]:
execute_instructions(Input(8), part_b=True)

5347

## [Day 9](http://adventofcode.com/2017/day/9): Stream Processing

In [75]:
def remove_ignored(inp):
    return re.sub(r'!.', '', inp)

In [76]:
def remove_garbage(inp):
    return re.sub(r'<[^>]*?>', '', inp)

In [77]:
def count_garbage(inp):
    garbage = 0
    garbage_location = re.search(r'<[^>]*?>', inp)
    while garbage_location:
        end = garbage_location.end()
        start = garbage_location.start()
        length = end - start
        garbage += (length - 2) # Remove the leading < and trailing >
        inp = inp[:start] + inp[end:]
        garbage_location = re.search(r'<[^>]*?>', inp)
    return garbage

In [78]:
def score_brackets(inp):
    group_value = 0
    score = 0
    for symbol in inp:
        if symbol == '{':
            group_value += 1
        if symbol == '}': 
            score += group_value
            group_value -= 1
    return score

In [79]:
in9 = Input(9).read()

In [80]:
in9 = remove_ignored(in9)

In [81]:
count_garbage(in9)

5547

In [82]:
in9 = remove_garbage(in9)

In [83]:
score_brackets(in9)

10820

In [84]:
def score_brackets_parse(stream):
    group_value, total_score = 0, 0
    garbage_count, in_garbage = 0, False
    
    i = 0
    while i < len(stream):
        symbol = stream[i]
        if symbol == '!': i += 1
        elif in_garbage:
            if symbol == '>': in_garbage = False
            else: garbage_count += 1
        elif symbol == '<': in_garbage = True
        elif symbol == '>': in_garbage = False
        elif symbol == '{': group_value += 1
        elif symbol == '}': 
            total_score += group_value
            group_value -= 1
        i += 1
    return {'score': total_score, 'garbage': garbage_count}

In [182]:
in9 = Input(9).read()
score_brackets_parse(in9)

{'garbage': 5547, 'score': 10820}

## [Day 10](http://adventofcode.com/2017/day/10): Knot Hash

In [86]:
def solve10(inp, part_b=False):
    if part_b:
        inp = list(map(ord, inp))
        inp += [17, 31, 73, 47, 23]
        rounds = 64
    else:
        inp = [int(x) for x in inp.split(',')]
        rounds = 1
        
    hashes = list(range(256))    
    current_position = 0
    skip = 0
    
    for _ in range(rounds):
        for length in inp:
            if length <= len(hashes):
                # Rotate the array so the current position is index 0
                # This stops us from having to deal with wrapping around the edges
                hashes = rotate(hashes, current_position)
                hashes[:length] = list(reversed(hashes[:length]))
                # Then rotate back!
                hashes = rotate(hashes, -current_position)
            current_position += length + skip
            skip += 1 
            
    if part_b:
        final_hash = ''
        for start in range(0, len(hashes), 16):
            part = hashes[start:start+16]
            hashp = reduce(operator.xor, part)
            final_hash += f'{hashp:02x}'
        return final_hash
    else:
        return hashes[0] * hashes[1]

In [183]:
solve10(Input(10).read().strip())

2928

In [184]:
solve10(Input(10).read().strip(), part_b=True)

'0c2f794b2eb555f7830766bf8fb65a16'